In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/godel-data-2/test_medium.jsonl
/kaggle/input/godel-data-2/train_seq2seq.jsonl
/kaggle/input/godel-data-2/val_seq2seq.jsonl
/kaggle/input/godel-data-2/test_easy.jsonl
/kaggle/input/godel-data-2/test_hard.jsonl


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")


In [4]:
!pip install datasets


In [5]:
import unsloth
import torch
import json
from datasets import Dataset, load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

# === 1. Load Model and Tokenizer ===
max_seq_length = 4096
dtype = torch.float16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2", # Using the Unsloth optimized Mistral model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Apply Phi-3.5 native chat template
# tokenizer = get_chat_template(
#     tokenizer, 
#     chat_template = "phi-3.5"
# )


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-15 07:22:37.387754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755242557.763591      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755242557.868637      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.5: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

{"timestamp":"2025-08-15T07:23:42.954337Z","level":"WARN","fields":{"message":"Status Code: 504. Retrying...","request_id":""},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":236}
{"timestamp":"2025-08-15T07:23:42.954396Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 2.786074221s before the next attempt"},"filename":"/root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}


special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 96,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


Unsloth 2025.8.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
import json

# *** PROMPT TEMPLATE CHANGED FOR MISTRAL ***
# This new function parses your data and formats it with the correct [INST] tags.
def format_user_mcq_data_for_mistral(examples):
    """
    Formats your specific medical MCQ dataset for the Mistral Instruct chat template.
    """
    texts = []
    # System prompt provides the initial instruction
    system_prompt = "You are a helpful medical assistant that answers multiple choice questions accurately. Provide the correct letter option followed by the full text of that option."
    
    batch_size = len(examples['input'])
    for i in range(batch_size):
        try:
            full_input = examples['input'][i]
            full_output = examples['output'][i]

            input_parts = full_input.split('\n')
            question = input_parts[0].strip()
            options_str = "\n".join(input_parts[1:])
            assistant_response = full_output.replace(" is the correct answer.", "").strip()

            # Create the Mistral-formatted prompt string
            # The <s> and </s> tokens are beginning/end of sequence tokens.
            # [INST] and [/INST] are the instruction markers.
            prompt = f"<s>[INST] {system_prompt}\n\nQuestion: {question}\n\nOptions:\n{options_str}\n\nWhat is the correct answer? [/INST] {assistant_response}</s>"
            texts.append(prompt)

        except Exception as e:
            print(f"Error processing example {i}: {e}")
            texts.append("") # Append empty string on error

    return {"text": texts}

# Load your dataset using the 'json' format for .jsonl files
train_file_path = "/kaggle/input/godel-data-2/train_seq2seq.jsonl"
val_file_path = "/kaggle/input/godel-data-2/val_seq2seq.jsonl"

print(f"Loading training data from: {train_file_path}")
raw_train_dataset = load_dataset("json", data_files=train_file_path, split="train")

print(f"Loading validation data from: {val_file_path}")
raw_val_dataset = load_dataset("json", data_files=val_file_path, split="train")

# Apply the new formatting function
train_dataset = raw_train_dataset.map(
    format_user_mcq_data_for_mistral,
    batched=True,
    remove_columns=raw_train_dataset.column_names,
    desc="Formatting training examples"
)
val_dataset = raw_val_dataset.map(
    format_user_mcq_data_for_mistral,
    batched=True,
    remove_columns=raw_val_dataset.column_names,
    desc="Formatting validation examples"
)

# Filter out any examples that failed to parse
train_dataset = train_dataset.filter(lambda x: len(x["text"].strip()) > 0)
val_dataset = val_dataset.filter(lambda x: len(x["text"].strip()) > 0)

# *** DATASET SIZE LIMITED TO 20,000 EXAMPLES ***
if len(train_dataset) > 20000:
    train_dataset = train_dataset.select(range(20000))

print(f"\nFinal training examples: {len(train_dataset)}")
print(f"Final validation examples: {len(val_dataset)}")

print("\n=== Sample Formatted Example ===")
if len(train_dataset) > 0:
    print(train_dataset[0]["text"][:500] + "...")

Loading training data from: /kaggle/input/godel-data-2/train_seq2seq.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Loading validation data from: /kaggle/input/godel-data-2/val_seq2seq.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Formatting training examples:   0%|          | 0/161505 [00:00<?, ? examples/s]

Formatting validation examples:   0%|          | 0/3297 [00:00<?, ? examples/s]

Filter:   0%|          | 0/161505 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3297 [00:00<?, ? examples/s]


Final training examples: 20000
Final validation examples: 3297

=== Sample Formatted Example ===
<s>[INST] You are a helpful medical assistant that answers multiple choice questions accurately. Provide the correct letter option followed by the full text of that option.

Question: Glycogenolysis is mediated by:

Options:
A. Alpha 1 receptor
B. Alpha l+beta2 receptor
C. Beta 2 receptor
D. Alpha 2 + beta 1 receptor

What is the correct answer? [/INST] A. Alpha 1 receptor</s>...


In [8]:
print("\n=== Sample Formatted Example ===")
if len(train_dataset) > 0:
    print(train_dataset[0]["text"][:500] + "...")
    print(f"\nEOS token: {repr(tokenizer.eos_token)}")


=== Sample Formatted Example ===
<s>[INST] You are a helpful medical assistant that answers multiple choice questions accurately. Provide the correct letter option followed by the full text of that option.

Question: Glycogenolysis is mediated by:

Options:
A. Alpha 1 receptor
B. Alpha l+beta2 receptor
C. Beta 2 receptor
D. Alpha 2 + beta 1 receptor

What is the correct answer? [/INST] A. Alpha 1 receptor</s>...

EOS token: '</s>'


In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported



trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set to 1 since we're using a subset
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10, # Log loss every 10 steps
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        # *** EVALUATION STRATEGY UPDATED ***
        eval_strategy = "steps", # Evaluate during training
        eval_steps = 250,              # Evaluate every 250 steps
        save_strategy = "steps",       # Save checkpoint at same interval
        save_steps = 250,              # Save every 250 steps
        save_total_limit = 3,          # Only keep the last 3 checkpoints
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/20000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/3297 [00:00<?, ? examples/s]

In [12]:
# Start Training
print("Starting training...")
trainer_stats = trainer.train()
print("Training completed!")

# Save Model
model.save_pretrained("medical_mistral7b_lora")
tokenizer.save_pretrained("medical_mistral7b_lora")
print("Model saved successfully!")

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1 | Total steps = 1,250
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080 of 7,325,618,176 (1.15% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
250,0.839000,0.816462
500,0.754500,0.793674
750,0.727500,0.767383
1000,0.754900,0.742772
1250,0.743900,0.727826


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Training completed!
Model saved successfully!


In [15]:


model.push_to_hub("Tejas1615/medical-mstrl7b-lora_adapters_final_16b", token=secret_value_0)
tokenizer.push_to_hub("Tejas1615/medical-mstrl7b-lora_adapters_final_16b", token=secret_value_0)

README.md:   0%|          | 0.00/608 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Tejas1615/medical-mstrl7b-lora_adapters_final_16b


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]